In [ ]:
import json

# Carregar o arquivo JSON com os dados dos produtos
with open("../../data/processed/products_data.json", "r") as file:
    products = json.load(file)

# Inicializar o conjunto de treinamento
TRAINING_DATA = []

for product in products:
    # Extrair as informações necessárias com verificações
    name = product.get("productName", "Produto desconhecido")
    brand = product.get("brand", "Marca desconhecida")
    categories = product.get("categories", ["Categoria desconhecida"])

    # Verificar a presença do preço com segurança
    price = None
    if "sellers" in product and product["sellers"]:
        commertial_offer = product["sellers"][0].get("commertialOffer")
        if commertial_offer:
            installment = commertial_offer.get("Installment")
            if isinstance(installment, dict):  # Verifica se Installment é um dicionário
                price = installment.get("Value")

    # Verificar se productClusters existe e extrair seus valores, caso contrário usar um placeholder
    clusters = product.get("productClusters", {})
    cluster_text = ", ".join(clusters.values()) if clusters else "Nenhum cluster disponível"
    
    # Construir a frase para treinamento, incluindo todas as categorias e ajustando a presença de preço
    categories_text = ", ".join(categories)
    if price is not None:
        phrase = f"{name} da marca {brand}, nas categorias {categories_text}, classificado como {cluster_text}, em oferta por R$ {price:.2f}."
    else:
        phrase = f"{name} da marca {brand}, nas categorias {categories_text}, classificado como {cluster_text}."

    # Identificar as posições das entidades na frase
    entities = [
        (phrase.find(name), phrase.find(name) + len(name), "PRODUCT_NAME"),
        (phrase.find(brand), phrase.find(brand) + len(brand), "BRAND")
    ]
    
    # Adicionar cada categoria individualmente como uma entidade "CATEGORY"
    start_index = phrase.find("nas categorias") + len("nas categorias ") + 1
    for category in categories:
        category_start = phrase.find(category, start_index)
        if category_start != -1:
            entities.append((category_start, category_start + len(category), "CATEGORY"))
        start_index = category_start + len(category)

    # Adicionar cada cluster como uma entidade "CLUSTER"
    cluster_start = phrase.find(cluster_text)
    if cluster_start != -1:
        entities.append((cluster_start, cluster_start + len(cluster_text), "CLUSTER"))
    
    # Adicionar a entidade PRICE se o preço estiver presente
    if price is not None:
        price_text = f"R$ {price:.2f}"
        price_start = phrase.find(price_text)
        entities.append((price_start, price_start + len(price_text), "PRICE"))

    # Adicionar a frase processada ao conjunto de dados de treinamento
    TRAINING_DATA.append((phrase, {"entities": entities}))

# Salvar o conjunto de treinamento em um arquivo JSON
with open("../../model/tratraining_data/training_data.json", "w") as out_file:
    json.dump(TRAINING_DATA, out_file, indent=4, ensure_ascii=False)

print("Arquivo de treinamento 'training_data.json' gerado com sucesso.")
